In [ ]:
# chèn các thư viện cần thiết
import time
import math
import cv2
import numpy as np
import matplotlib.pyplot as plt
import csv
from google.colab.patches import cv2_imshow
confid = 0.5
thresh = 0.5
anpha=50
beta=80
# vname=""
vname=input("Video name in videos folder:  ")
vid_path = "/content/drive/MyDrive/BTL_AI_Xác định khoảng cách covid 19_Nhóm 15/videos/"+vname
angle_factor = 0.8

# Hàm tính khoảng cách giữa 2 pixel theo công thưc Euclide
def dist(c1, c2):
    return ((c1[0] - c2[0]) ** 2 + (c1[1] - c2[1]) ** 2) ** 0.5
# hàm kiểm tra khoảng cách giữa 2 người

def isclose1(p1,p2):
    c_d = dist(p1[2], p2[2])
    if 0<c_d<anpha:
        return (1,c_d)
    elif anpha<c_d<beta:
        return (2,c_d)
    else:
        return (0,c_d)

#  lấy đường dấn file coco.name sau đó đọc file gán vào mảng LABELS 

labelsPath = "/content/drive/MyDrive/BTL_AI_Xác định khoảng cách covid 19_Nhóm 15/coco.names"
LABELS = open(labelsPath).read().strip().split("\n")

# np.random.seed(42)
# Lấy đường dẫn file weight và config

weightsPath = "/content/drive/MyDrive/BTL_AI_Xác định khoảng cách covid 19_Nhóm 15/yolov3.weights"
configPath = "/content/drive/MyDrive/BTL_AI_Xác định khoảng cách covid 19_Nhóm 15/yolov3.cfg"

# Đọc mô hình mạng 
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
# Lưu các lớp của mô hình mạng
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
FR=0
vs = cv2.VideoCapture(vid_path)
# vs = cv2.VideoCapture(0)  ## Nếu muốn sử dụng webcam
writer = None

(W, H) = (None, None)


while True:
#  Đọc video
    (grabbed, frame) = vs.read()
    # Kiểm tra điều kiện đọc video
    if not grabbed:
        break

    if W is None or H is None:
      # Lưu kích thước khung vào (H,W)
        (H, W) = frame.shape[:2] 
        FW=W
        if(W<1075):
            FW = 1075
        FR = np.zeros((H+210,FW,3), np.uint8)

        col = (255,255,255)
        FH = H + 210
    FR[:] = col

# Tiền xử lí ảnh đầu vào :scale ảnh ,resize, chuyển BGR-> RGB 

    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
                                 swapRB=True, crop=False)
    # Phát hiện đối tượng 
    net.setInput(blob)
    start = time.time()
    # các lớp của mạng
    layerOutputs = net.forward(ln)
    end = time.time()
# Tạo mảng lưu trữ các box, độ tin cậy, ID của việc dự đoán 
    boxes = []
    confidences = []
    classIDs = []
# Vòng lặp duyêt qua tất cả các lớp trong mô hình mạng
    for output in layerOutputs:
      # Duyệt qua các phát hiện của đầu ra mỗi lớp
        for detection in output:
            #  Trích xuất ID và độ tin cậy của đối tượng    
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            # chỉ xét các phát hiện là người
            if LABELS[classID] == "person":
                # Tạo box nếu độ tin cậy > confid
                if confidence > confid:
                  # Tính tọa độ box rồi trả vể tọa độ tâm chiều cao, chiều rộng
                    box = detection[0:4] * np.array([W, H, W, H])
                    (centerX, centerY, width, height) = box.astype("int")
                    # xác định tọa độ trên cùng bên trái của box
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))
                    # Cập nhật danh sách box, độ tin cậy, ID
                    boxes.append([x, y, int(width), int(height)])
                    confidences.append(float(confidence))
                    classIDs.append(classID)
    # Loại bỏ các box có nmsthresh < thresh để tránh sự chồng chéo
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, confid, thresh)
    #  Điều kiện có it nhất một phát hiện
    if len(idxs) > 0:
        # các mảng lưu trữ các trạng thái thông tin .
        status = []
        close_pair = []
        s_close_pair = []
        center = []
        co_info = []
        # Làm phẳng idxs
        idf = idxs.flatten()
        # Duyệt qua tất cả các thành phần idf
        for i in idf:
            # trích xuất tọa độ box
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            # tọa độ trung tâm
            cen = [int(x + w / 2), int(y + h / 2)]
            center.append(cen)
            # tạo chấm trong tại vị trí trung tâm rồi lưu thông tin
            cv2.circle(frame, tuple(cen),1,(0,0,0),1)
            co_info.append([w, h, cen])

            status.append(0)

        # Duyệt tất cả các tọa độ có người
        for i in range(len(center)):
            for j in range(i+1,len(center)):
              # Kiểm tra khoảng cách giữa hai người
                g = isclose1(co_info[i],co_info[j])
              
              #  nếu g==1 thì rủi ro cao
                if g[0] == 1:
                    close_pair.append([center[i], center[j],g[1]])
                    status[i] = 1
                    status[j] = 1
              #  nếu g==2 thì rủi ro thấp
                elif g[0] == 2:
                    s_close_pair.append([center[i], center[j],g[1]])
                    if status[i] != 1:
                        status[i] = 2
                    if status[j] != 1:
                        status[j] = 2
              #  nếu g==0 thì an toàn
        # Cập nhật số trạng thái của mọi người
        total_p = len(center)
        low_risk_p = status.count(2)
        high_risk_p = status.count(1)
        safe_p = status.count(0)
        kk = 0
        # Duyêt qua tất cả idf lần nũa
        for i in idf:
          # Ghi chú các thông tin tại dưới mỗi kkhung
            cv2.line(FR,(0,H+1),(FW,H+1),(0,0,0),2)
            cv2.putText(FR, "Phan tich khoang cach xa hoi. COVID-19", (210, H+60),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
            cv2.rectangle(FR, (20, H+80), (510, H+180), (100, 100, 100), 2)
            cv2.putText(FR, "Nhung duong ket noi 2 nguoi ", (30, H+100),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (100, 100, 0), 2)
            cv2.putText(FR, "-- Vang: CLOSE", (50, H+90+40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 170, 170), 2)
            cv2.putText(FR, "-- Do: VERY CLOSE", (50, H+40+110),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            
            cv2.rectangle(FR, (535, H+80), (1060, H+140+40), (100, 100, 100), 2)
            cv2.putText(FR, "Bounding boxes gioi han muc do rui do .", (545, H+100),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (100, 100, 0), 2)
            cv2.putText(FR, "-- Do:Cao", (565, H+90+40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 150), 2)
            cv2.putText(FR, "--   Cam:Thap", (565, H+150),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 120, 255), 2)

            cv2.putText(FR, "--    Xanh: An toan", (565, H+170),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 150, 0), 2)

            
            tot_str = "Tong: " + str(total_p)
            high_str = "Rui ro cao: " + str(high_risk_p)
            low_str = "Rui ro thap: " + str(low_risk_p)
            safe_str = "An toan: " + str(safe_p)

            cv2.putText(FR, tot_str, (10, H +25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
            cv2.putText(FR, safe_str, (200, H +25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 170, 0), 2)
            cv2.putText(FR, low_str, (380, H +25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 120, 255), 2)
            cv2.putText(FR, high_str, (630, H +25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 150), 2)

            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            # vẽ các boundind box cho người với các trạng thái với các màu khác nhau
            if status[kk] == 1:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 150), 2)

            elif status[kk] == 0:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            else:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 120, 255), 2)

            kk += 1
        # vẽ đường line nối các cặp gần nhau ở mức rủi ro cao và rủi ro thấp
        for h in close_pair:
            (x1,y1)=h[0]
            (x2,y2)=h[1]
            (x3,y3)=(int((h[0][0]+h[1][0])/2),int((h[0][1]+h[1][1])/2))
            cv2.line(frame, (x1,y1), (x2,y2), (0, 0, 255), 2)
            cv2.putText(frame, str(round(h[2])),(x3+2,y3+4),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
            
            
        for h in s_close_pair:
            (x1,y1)=h[0]
            (x2,y2)=h[1]
            (x3,y3)=(int((h[0][0]+h[1][0])/2),int((h[0][1]+h[1][1])/2))
            cv2.line(frame, (x1,y1), (x2,y2), (0, 255, 255), 2)
            cv2.putText(frame, str(round(h[2])),(x3+2,y3+4),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
        FR[0:H, 0:W] = frame
        frame = FR
        # in ra khung hình được xử lí
        cv2_imshow(frame)
        cv2.waitKey(1)
  # Ghi lại các khung thành video đầu ra
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter("outputvideo.avi", fourcc, 30,
                                 (frame.shape[1], frame.shape[0]), True)

    writer.write(frame.astype('uint8'))
print("Processing finished: open"+"out"+vname)
# Giải phóng quá trình ghi và đọc video
writer.release()
vs.release()
